In [1]:
import pandas as pd
import numpy as np

In [2]:
from tg.datasets import DatasetFactoryLookupCallback
from tg.model_interactor import ModelInteractor
from tg.splitters import AnchoredSplitter

In [3]:
dataset_name = 'AIR_PASSENGERS'
data_factory = DatasetFactoryLookupCallback(dataset_name=dataset_name)

In [4]:
model_name = 'STL'
params = {}

In [5]:
y, X = data_factory(model_name=model_name)

mi = ModelInteractor(model_name=model_name)
mi.load(y=y,
        X=X,
        dataset_name=data_factory.dataset_name,
        timesteps=data_factory.dataset.period,
        train_size=data_factory.dataset.train_size)

trains, test = mi.split_trains_test(
    y=mi.y,
    splitter_class=AnchoredSplitter,
    splitter_args={'min_train_points': data_factory.dataset.train_size},
    X=mi.X)

preds = mi.fit_predict(trains=trains, test=test, parameters=params)
metrics = mi.evaluate(preds=preds, test=test)

100%|██████████| 34/34 [00:00<00:00, 86.66it/s]


In [6]:
mi.tune_hyperparameters(splitter_class=AnchoredSplitter,
                        splitter_args={
                            'min_train_points':
                            data_factory.dataset.tuning_train_size
                        },
                        n_trials=10)

[I 2022-10-27 02:35:40,803] A new study created in RDB with name: AIR_PASSENGERS/STL/v1
100%|██████████| 104/104 [00:00<00:00, 124.82it/s]
[I 2022-10-27 02:35:41,808] Trial 0 finished with value: 14.627151777418769 and parameters: {'seasonal': 15, 'seasonal_deg': 1, 'trend_deg': 1, 'low_pass_deg': 1, 'seasonal_jump': 6, 'trend_jump': 9, 'low_pass_jump': 7, 'robust': False}. Best is trial 0 with value: 14.627151777418769.
100%|██████████| 104/104 [00:01<00:00, 64.06it/s]
[I 2022-10-27 02:35:43,550] Trial 1 finished with value: 15.337700145723254 and parameters: {'seasonal': 45, 'seasonal_deg': 1, 'trend_deg': 1, 'low_pass_deg': 1, 'seasonal_jump': 8, 'trend_jump': 1, 'low_pass_jump': 7, 'robust': True}. Best is trial 0 with value: 14.627151777418769.
100%|██████████| 104/104 [00:00<00:00, 127.97it/s]
[I 2022-10-27 02:35:44,476] Trial 2 finished with value: 32.6452531737919 and parameters: {'seasonal': 93, 'seasonal_deg': 0, 'trend_deg': 1, 'low_pass_deg': 0, 'seasonal_jump': 7, 'trend_j

{'low_pass_deg': 1,
 'low_pass_jump': 7,
 'robust': False,
 'seasonal': 15,
 'seasonal_deg': 1,
 'seasonal_jump': 6,
 'trend_deg': 1,
 'trend_jump': 9}

In [7]:
metrics

{'rmse': 15.922787718567635,
 'smape': 1.5067592761444077,
 'mape': 3.020717144866504,
 'mae': 12.919115914763093}